**Задание 1** В зале суда есть 5 присяжных, каждый из них по отдельности с вероятностью 70% может правильно определить виновен подсудимый или нет. С какой вероятностью они все вместе вынесут правильный вердикт, если решение принимается большинством голосов?
- 70.00%
- 83.20%
- 83.70%
- 87.50%

Решение:
поскольку большинство голосов – 3, тогда наше m = 3, N = 5, p = 0.7. Подставляем в формулу из статьи $$ \large \mu = \sum_{i=3}^{5}C_5^i0.7^i(1-0.7)^{5-i} $$
После подставления и проделывания всех операций получим ответ 83.70%

In [1]:
%pylab inline
# отключим предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Сделаем функцию, которая будет заменять NaN значения на медиану в каждом столбце таблицы 
def delete_nan(table):
    for col in table.columns:
        table[col]= table[col].fillna(table[col].median())
    return table   

## Считываем данные
data = pd.read_csv('credit_scoring_train.csv', sep =';')

independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']

table =delete_nan(data)

X =table[independent_columns_names]
y = table['SeriousDlqin2yrs']

Populating the interactive namespace from numpy and matplotlib


# Бутстрэп

**Задание 2.** Сделайте интервальную оценку среднего возраста (age) для клиентов, которые просрочили выплату кредита, с 90% "уверенностью". (используйте пример из статьи. Поставьте np.random.seed(0) как это сделано в статье)

In [2]:
def get_bootstrap_samples(data, n_samples):
    # функция для генерации подвыборок с помощью бутстрэпа
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    # функция для интервальной оценки
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

# сохранение в отдельные numpy массивы данных по просрочке
churn = data[data['SeriousDlqin2yrs'] == 1]['age'].values

# ставим seed для воспроизводимости результатов
np.random.seed(0)

# генерируем выборки с помощью бутстрэра и сразу считаем по каждой из них среднее
churn_mean_scores = [np.mean(sample) 
                       for sample in get_bootstrap_samples(churn, 1000)]

#  выводим интервальную оценку среднего
print("Mean interval",  stat_intervals(churn_mean_scores, 0.1))

Mean interval [ 45.71379414  46.12700479]


# Логистическая регрессия

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

lr = LogisticRegression(random_state=5, class_weight= 'balanced')
parameters = {'C': (0.0001, 0.001, 0.01, 0.1, 1, 10)}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

**Задание 3.**
Сделайте GridSearch с метрикой "roc-auc" по параметру C. Какое оптимальное значение параметра С?



In [4]:
grid_search = GridSearchCV(lr, parameters, n_jobs=-1, scoring ='roc_auc', cv=skf)
grid_search = grid_search.fit(X, y)
grid_search.best_estimator_

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=5,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

**Задание 4.** 
Можно ли считать лучшую модель устойчивой? (модель считаем устойчивой, если стандартное отклонение на валидации меньше 0.5%)

In [5]:
grid_search.cv_results_['std_test_score'][1]

0.0037606335564454828

In [6]:
grid_search.best_score_

0.79602038989281942

### Определение влияния признаков

**Задание 5.** Определите самый важный признак. Важность признака определяется абсолютным значением его коэффициента. Так же нужно нормализировать все признаки, что бы можно их было корректно сравнить.

In [7]:
from sklearn.preprocessing import StandardScaler
lr = LogisticRegression(C=0.001,random_state=5, class_weight= 'balanced')
scal = StandardScaler()
lr.fit(scal.fit_transform(X), y)
for f, c in zip(independent_columns_names, lr.coef_[0]):
    print(f,c,sep="     ")

age     -0.416303678767
NumberOfTime30-59DaysPastDueNotWorse     0.7240043159
DebtRatio     -0.0240818648119
NumberOfTimes90DaysLate     0.517672915413
NumberOfTime60-89DaysPastDueNotWorse     0.194732165768
MonthlyIncome     -0.162863529686
NumberOfDependents     0.101326025081


Самый важный признак – NumberOfTime30-59DaysPastDueNotWorse

**Задание 6.** Посчитайте долю влияния `DebtRatio` на предсказание. (Реализуйте функцию [softmax](https://en.wikipedia.org/wiki/Softmax_function))

In [8]:
print((np.exp(lr.coef_[0]) / np.sum(np.exp(lr.coef_[0])))[2])

0.114205367199


**Задание 7.** 
Давайте посмотрим как можно интерпретировать влияние наших признаков. Для этого заного оценим логистическую регрессию в абсолютных величинах. После этого посчитайте во сколько раз увеличатся шансы, что клиент не выплатит кредит, если увеличить возраст на 20 лет при всех остальных равных значениях признаков. (теоретический расчет можно посмотреть [здесь](https://www.unm.edu/~schrader/biostat/bio2/Spr06/lec11.pdf))

In [9]:
lr = LogisticRegression(C=0.001,random_state=5, class_weight= 'balanced')
lr.fit(X, y)
for f, c in zip(independent_columns_names, lr.coef_[0]):
    print(f,c,sep="     ")

age     -0.0181852818846
NumberOfTime30-59DaysPastDueNotWorse     0.482349233865
DebtRatio     -1.07983115273e-05
NumberOfTimes90DaysLate     0.430314155406
NumberOfTime60-89DaysPastDueNotWorse     0.0659578713554
MonthlyIncome     -1.14476586003e-05
NumberOfDependents     0.11535624898


In [10]:
np.exp(lr.coef_[0][0]*20)

0.6950957746271984

$\exp^{\beta\delta}$ – во столько раз больше шансы, что клиент не выплатит кредит. Где $\delta$ – на сколько делаем прирост. Например, если увеличить возраст на 20 лет, то шансы, что человек не выплатит в кредит увеличатся в 0.69.

# Случайный лес

In [11]:
from sklearn.ensemble import RandomForestClassifier

# Инициализируем случайный лес с 100 деревьями и сбалансированными классами 
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42, oob_score=True, class_weight='balanced')

## Будем искать лучшие параметры среди следующего набора
parameters = {'max_features': [1, 2, 4], 'min_samples_leaf': [3, 5, 7, 9], 'max_depth': [5,10,15]}

## Делаем опять же стрэтифайд k-fold валидацию. Инициализация которой должна у вас продолжать храниться в skf

**Задание 8.** На сколько точность лучшей модели случайного леса выше точности логистической регрессии на валидации?

In [12]:
# Ответ 5
rf_grid_search = GridSearchCV(rf, parameters, n_jobs=-1, scoring ='roc_auc', cv=skf)
rf_grid_search = rf_grid_search.fit(X, y)
print(rf_grid_search.best_score_ - grid_search.best_score_)

0.0396110583184


**Задание 9.** Определите какой признак имеет самое слабое влияние.

In [13]:
independent_columns_names[argmin(rf_grid_search.best_estimator_.feature_importances_)]

'NumberOfDependents'

** Задание 10.** Какое наиболее существенное примущество логистической регрессии перед случайным лесом для нашей бизнес-задачи?

- меньше тратится времени для тренировки модели;
- меньше параметров для перебора;
- интепретируемость признаков;
- линейные свойства алгоритма.

В итоге мы получили, что алгоритм случайно леса лучше сработал для нашей задачи скоринга. Точность случайного леса почти на 4% выше. Причинами такого результата стали – небольшое количество признаков и свойства случайного леса, как композиции.

Но преимущество логистической регрессии в том, что мы можем проинтерпретировать влияние коэффициентов на результат.

# Бэггинг

In [14]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
parameters = {'max_features': [2, 3, 4], 'max_samples': [0.5, 0.7, 0.9], "base_estimator__C": [0.0001, 0.001, 0.01, 1, 10, 100]}

**Задание 11.** Следующая задача обучить бэггинг классификатор. В качестве базовых классификаторов возьмите 100 логистических регрессий и на этот раз используйте не GridSearchCV, а RandomizedSearchCV. Так как перебирать все 54 варианта комбинаций долго, то поставьте максимальное число итераций 20 для RandomizedSearchCV. Так же не забудьте передать параметр валидации cv и random_state=1. Какая лучшая точность получилась?

In [15]:
bg = BaggingClassifier(LogisticRegression(class_weight= 'balanced'),n_estimators=100, n_jobs=-1, random_state=42)
r_grid_search = RandomizedSearchCV(bg, parameters, n_jobs=-1, scoring ='roc_auc', cv=skf, n_iter=20, random_state=1)
r_grid_search = r_grid_search.fit(X, y)

In [16]:
r_grid_search.best_score_

0.80746778722957357

**Задача 12.** Дайте интерпретацию лучших параметров для бэггинга. Почему именно такие значения оказались лучшими?

- для бэггинга важно использовать как можно меньше признаков
- бэггинг лучше работает на небольших выборках
- меньше корреляция между одиночными моделями
- чем больше признаков, тем меньше теряется информации

Преимущество случайного леса в том, что деревья в композиции не коррелируют между собой и это дает максимальную пользу. Аналогично и для бэггинга с логистической регрессией, чем слабее коррелируют между собой одиночные модели, тем выше точность. Поскольку в логистической регрессии практически нет случайности, то мы должны менять наборы признаков и объектов для достижении максимальной декоррелированости наших моделей.